In [1]:
# Path to source code
%cd ../../

c:\Users\ajaoo\Desktop\Projects\Multivate-forecasting


In [2]:
# Imports for handling data
import os
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle
# from sklearn.model_selection import TimeSeriesSplit, train_test_split

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
# from sklearn.linear_model import LinearRegression
# from scipy.stats import spearmanr

# Imports for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# Progress bar
from tqdm.autonotebook import tqdm
# Enable progress apply for pandas
tqdm.pandas()


# Local imports for data loaders and models
from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel, Seq2SeqConfig, Seq2SeqModel, RNNConfig
from src.transforms.target_transformations import AutoStationaryTransformer


pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    
torch.set_float32_matmul_precision('high')

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings

warnings.filterwarnings("ignore")

# %load_ext tensorboard

Seed set to 42


In [3]:
def format_plot(
    fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15
):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig


def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


def plot_forecast(
    pred_df, forecast_columns, forecast_display_names=None, save_path=None
):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.Set2  # Using a different color palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines",
            marker=dict(size=6, opacity=0.5),
            line=dict(color=act_color, width=2),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=4),
                line=dict(color=next(colors), width=2),
                name=display_col,
            )
        )

    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [4]:
# Load and Prepare Data
data_path = Path("data/processed/merged_nhs_covid_data.csv")
data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data["date"] = pd.to_datetime(data["date"])

In [5]:
# check the unique values in the areaName column
data["areaName"].unique()

array(['East of England', 'London', 'Midlands',
       'North East and Yorkshire', 'North West', 'South East',
       'South West'], dtype=object)

In [6]:
# Select a different area name
selected_area = "Midlands"  # "London", "South East", "North West", "East of England", "South West", "West Midlands", "East Midlands", "Yorkshire and The Humber", "North East"
data_filtered = data[data["areaName"] == selected_area]

# Data Processing
data_filtered["date"] = pd.to_datetime(data_filtered["date"])
data_filtered.sort_values(by=["date", "areaName"], inplace=True)
data_filtered.drop(
    [
        "areaName",
        "areaCode",
        "cumAdmissions",
        "cumulative_confirmed",
        "cumulative_deceased",
        "population",
        "latitude",
        "longitude",
        "epi_week",
    ],
    axis=1,
    inplace=True,
)

In [7]:
def add_rolling_features(df, window_size, columns, agg_funcs=None):
    if agg_funcs is None:
        agg_funcs = ["mean"]
    added_features = {}
    for column in columns:
        for func in agg_funcs:
            roll_col_name = f"{column}_rolling_{window_size}_{func}"
            df[roll_col_name] = df[column].rolling(window_size).agg(func)
            if column not in added_features:
                added_features[column] = []
            added_features[column].append(roll_col_name)
    # Drop rows with NaN values which are the result of rolling window
    df.dropna(inplace=True)
    return df, added_features


# Configuration
window_size = 7
columns_to_roll = ["hospitalCases", "newAdmissions", "new_confirmed", "new_deceased"]
agg_funcs = ["mean", "std"]

# Apply rolling features for each column
data_filtered, added_features = add_rolling_features(
    data_filtered, window_size, columns_to_roll, agg_funcs
)

# Print added features for each column
for column, features in added_features.items():
    print(f"{column}: {', '.join(features)}")

hospitalCases: hospitalCases_rolling_7_mean, hospitalCases_rolling_7_std
newAdmissions: newAdmissions_rolling_7_mean, newAdmissions_rolling_7_std
new_confirmed: new_confirmed_rolling_7_mean, new_confirmed_rolling_7_std
new_deceased: new_deceased_rolling_7_mean, new_deceased_rolling_7_std


In [8]:
# Define a function to add time-lagged features to the dataset
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f"_lag_{lag}"
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]

data_filtered, added_features = add_lags(data_filtered, lags, ["covidOccupiedMVBeds"])
data_filtered.dropna(inplace=True)

In [9]:
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data_filtered = create_temporal_features(data_filtered, "date")
data_filtered.head()

,date,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,hospitalCases_rolling_7_mean,hospitalCases_rolling_7_std,newAdmissions_rolling_7_mean,newAdmissions_rolling_7_std,...,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
3552,2020-04-14,467.0,3345.0,359,330.0,82.0,3368.285714,49.895605,332.142857,22.915996,...,467.0,485.0,485.0,475.0,469.0,445.0,360.0,4,14,1
3549,2020-04-15,472.0,3101.0,242,216.0,67.0,3326.142857,110.437657,318.857143,40.891843,...,467.0,467.0,485.0,475.0,469.0,450.0,360.0,4,15,2
3550,2020-04-15,472.0,3101.0,242,394.0,108.0,3279.142857,127.553164,310.142857,50.260749,...,472.0,467.0,467.0,485.0,475.0,450.0,406.0,4,15,2
3547,2020-04-16,474.0,3095.0,283,266.0,31.0,3231.285714,124.323120,307.285714,51.292346,...,472.0,472.0,467.0,485.0,475.0,465.0,406.0,4,16,3
3548,2020-04-16,474.0,3095.0,283,514.0,89.0,3199.714286,127.251345,300.142857,50.591266,...,474.0,472.0,472.0,467.0,485.0,465.0,419.0,4,16,3


In [10]:
data_filtered["date"] = pd.to_datetime(data_filtered["date"])
data_filtered = data_filtered.set_index("date")
data_filtered.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,hospitalCases_rolling_7_mean,hospitalCases_rolling_7_std,newAdmissions_rolling_7_mean,newAdmissions_rolling_7_std,new_confirmed_rolling_7_mean,...,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2020-04-14,467.0,3345.0,359,330.0,82.0,3368.285714,49.895605,332.142857,22.915996,245.000000,...,467.0,485.0,485.0,475.0,469.0,445.0,360.0,4,14,1
2020-04-15,472.0,3101.0,242,216.0,67.0,3326.142857,110.437657,318.857143,40.891843,239.571429,...,467.0,467.0,485.0,475.0,469.0,450.0,360.0,4,15,2
2020-04-15,472.0,3101.0,242,394.0,108.0,3279.142857,127.553164,310.142857,50.260749,265.142857,...,472.0,467.0,467.0,485.0,475.0,450.0,406.0,4,15,2
2020-04-16,474.0,3095.0,283,266.0,31.0,3231.285714,124.323120,307.285714,51.292346,268.142857,...,472.0,472.0,467.0,485.0,475.0,465.0,406.0,4,16,3
2020-04-16,474.0,3095.0,283,514.0,89.0,3199.714286,127.251345,300.142857,50.591266,317.142857,...,474.0,472.0,472.0,467.0,485.0,465.0,419.0,4,16,3


In [11]:
# Set the target variable
target = "covidOccupiedMVBeds"

seasonal_period = 7
auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period)

# Fit and transform the target column to make it stationary
data_stat = auto_stationary.fit_transform(data_filtered[[target]], freq="D")

# Replace the original target values with the transformed stationary values
data_filtered[target] = data_stat.values

# Print the transformed data to check
data_filtered.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,hospitalCases_rolling_7_mean,hospitalCases_rolling_7_std,newAdmissions_rolling_7_mean,newAdmissions_rolling_7_std,new_confirmed_rolling_7_mean,...,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2020-04-14,32.840992,3345.0,359,330.0,82.0,3368.285714,49.895605,332.142857,22.915996,245.000000,...,467.0,485.0,485.0,475.0,469.0,445.0,360.0,4,14,1
2020-04-15,33.018215,3101.0,242,216.0,67.0,3326.142857,110.437657,318.857143,40.891843,239.571429,...,467.0,467.0,485.0,475.0,469.0,450.0,360.0,4,15,2
2020-04-15,33.018215,3101.0,242,394.0,108.0,3279.142857,127.553164,310.142857,50.260749,265.142857,...,472.0,467.0,467.0,485.0,475.0,450.0,406.0,4,15,2
2020-04-16,33.090707,3095.0,283,266.0,31.0,3231.285714,124.323120,307.285714,51.292346,268.142857,...,472.0,472.0,467.0,485.0,475.0,465.0,406.0,4,16,3
2020-04-16,33.090707,3095.0,283,514.0,89.0,3199.714286,127.251345,300.142857,50.591266,317.142857,...,474.0,472.0,472.0,467.0,485.0,465.0,419.0,4,16,3


In [12]:
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1763 entries, 2020-04-14 to 2022-09-12
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   covidOccupiedMVBeds           1763 non-null   float64
 1   hospitalCases                 1763 non-null   float64
 2   newAdmissions                 1763 non-null   int64  
 3   new_confirmed                 1763 non-null   float64
 4   new_deceased                  1763 non-null   float64
 5   hospitalCases_rolling_7_mean  1763 non-null   float64
 6   hospitalCases_rolling_7_std   1763 non-null   float64
 7   newAdmissions_rolling_7_mean  1763 non-null   float64
 8   newAdmissions_rolling_7_std   1763 non-null   float64
 9   new_confirmed_rolling_7_mean  1763 non-null   float64
 10  new_confirmed_rolling_7_std   1763 non-null   float64
 11  new_deceased_rolling_7_mean   1763 non-null   float64
 12  new_deceased_rolling_7_std    1763 non-null 

In [13]:
# Get the minimum and maximum date from the data
min_date = data_filtered.index.min()
max_date = data_filtered.index.max()
# Calculate the range of dates
date_range = max_date - min_date
print(f"Data ranges from {min_date} to {max_date} ({date_range.days} days)")

Data ranges from 2020-04-14 00:00:00 to 2022-09-12 00:00:00 (881 days)


In [14]:
# Filter data between the specified dates
start_date = "2020-04-14"
end_date = "2021-12-30"
data_filtered = data_filtered[start_date:end_date]

In [15]:
# selecting 1 year data for training and 2 months data for validation and 3 months data for testing
# Calculate the date ranges for train, val, and test sets
train_end = min_date + pd.Timedelta(days=date_range.days * 0.45)
val_end = train_end + pd.Timedelta(days=date_range.days * 0.15)

# Split the data into training, validation, and testing sets
train = data_filtered[data_filtered.index <= train_end]
val = data_filtered[(data_filtered.index > train_end) & (data_filtered.index < val_end)]
test = data_filtered[data_filtered.index > val_end]

# Calculate the percentage of dates in each dataset
total_sample = len(data_filtered)
train_sample = len(train) / total_sample * 100
val_sample = len(val) / total_sample * 100
test_sample = len(test) / total_sample * 100

print(
    f"Train: {train_sample:.2f}%, Validation: {val_sample:.2f}%, Test: {test_sample:.2f}%"
)
print(
    f"Train: {len(train)} samples, Validation: {len(val)} samples, Test: {len(test)} samples"
)
print(
    f"Max date in train: {train.index.max()}, Min date in train: {train.index.min()}, Max date in val: {val.index.max()}, Min date in val: {val.index.min()}, Max date in test: {test.index.max()}, Min date in test: {test.index.min()}"
)

Train: 63.39%, Validation: 21.10%, Test: 15.51%
Train: 793 samples, Validation: 264 samples, Test: 194 samples
Max date in train: 2021-05-15 00:00:00, Min date in train: 2020-04-14 00:00:00, Max date in val: 2021-09-24 00:00:00, Min date in val: 2021-05-16 00:00:00, Max date in test: 2021-12-30 00:00:00, Min date in test: 2021-09-25 00:00:00


In [16]:
train_dates = (train.index.min(), train.index.max())
val_dates = (val.index.min(), val.index.max())
test_dates = (test.index.min(), test.index.max())

print(f"Train dates: {train_dates}, Val dates: {val_dates}, Test dates: {test_dates}")



Train dates: (Timestamp('2020-04-14 00:00:00'), Timestamp('2021-05-15 00:00:00')), Val dates: (Timestamp('2021-05-16 00:00:00'), Timestamp('2021-09-24 00:00:00')), Test dates: (Timestamp('2021-09-25 00:00:00'), Timestamp('2021-12-30 00:00:00'))


In [17]:
# # check that the train, val and test data are in the correct order
# assert train.index.max() < val.index.min() < test.index.min()

# # Define the features to use for training
# features = [
#     "hospitalCases",
#     "newAdmissions",
#     "new_confirmed",
#     "new_deceased",
#     "hospitalCases_rolling_7_mean",
#     "hospitalCases_rolling_7_std",
#     "newAdmissions_rolling_7_mean",
#     "newAdmissions_rolling_7_std",
#     "new_confirmed_rolling_7_mean",
#     "new_confirmed_rolling_7_std",
#     "new_deceased_rolling_7_mean",
#     "new_deceased_rolling_7_std",
#     "covidOccupiedMVBeds_lag_1",
#     "covidOccupiedMVBeds_lag_2",
#     "covidOccupiedMVBeds_lag_3",
#     "covidOccupiedMVBeds_lag_5",
#     "covidOccupiedMVBeds_lag_7",
#     "covidOccupiedMVBeds_lag_14",
#     "covidOccupiedMVBeds_lag_21",
#     "month",
#     "day",
#     "day_of_week",
# ]

# # Define the target column
# target = "covidOccupiedMVBeds"

# # Define the input sequence length
# input_seq_len = 21

# # Define the output sequence length
# output_seq_len = 1

# # Define the batch size
# batch_size = 64

# # Define the number of workers for the data loader
# num_workers = 2

# # Define the data module
# data_module = TimeSeriesDataModule(
#     train_df=train,
#     val_df=val,
#     test_df=test,
#     features=features,
#     target=target,
#     input_seq_len=input_seq_len,
#     output_seq_len=output_seq_len,
#     batch_size=batch_size,
#     num_workers=num_workers,
# )

# # Load the data
# data_module.setup()


In [18]:
# plot the train data
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=train.index, y=train["covidOccupiedMVBeds"], mode="lines", name="Train Data"
    )
)
fig.update_layout(
    title="Train Data for COVID-19 Occupied MV Beds in Midlands Region",
    xaxis_title="Date",
    yaxis_title="COVID-19 Occupied MV Beds",
    template="plotly_white",
)
fig.show()

In [19]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert all the feature columns to float32
for col in sample_df.columns:
    sample_df[col] = sample_df[col].astype("float32")

sample_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1251 entries, 2020-04-14 to 2021-12-30
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   covidOccupiedMVBeds           1251 non-null   float32
 1   hospitalCases                 1251 non-null   float32
 2   newAdmissions                 1251 non-null   float32
 3   new_confirmed                 1251 non-null   float32
 4   new_deceased                  1251 non-null   float32
 5   hospitalCases_rolling_7_mean  1251 non-null   float32
 6   hospitalCases_rolling_7_std   1251 non-null   float32
 7   newAdmissions_rolling_7_mean  1251 non-null   float32
 8   newAdmissions_rolling_7_std   1251 non-null   float32
 9   new_confirmed_rolling_7_mean  1251 non-null   float32
 10  new_confirmed_rolling_7_std   1251 non-null   float32
 11  new_deceased_rolling_7_mean   1251 non-null   float32
 12  new_deceased_rolling_7_std    1251 non-null 

In [20]:
sample_df.columns

Index(['covidOccupiedMVBeds', 'hospitalCases', 'newAdmissions',
       'new_confirmed', 'new_deceased', 'hospitalCases_rolling_7_mean',
       'hospitalCases_rolling_7_std', 'newAdmissions_rolling_7_mean',
       'newAdmissions_rolling_7_std', 'new_confirmed_rolling_7_mean',
       'new_confirmed_rolling_7_std', 'new_deceased_rolling_7_mean',
       'new_deceased_rolling_7_std', 'covidOccupiedMVBeds_lag_1',
       'covidOccupiedMVBeds_lag_2', 'covidOccupiedMVBeds_lag_3',
       'covidOccupiedMVBeds_lag_5', 'covidOccupiedMVBeds_lag_7',
       'covidOccupiedMVBeds_lag_14', 'covidOccupiedMVBeds_lag_21', 'month',
       'day', 'day_of_week'],
      dtype='object')

In [21]:
columns_to_select = [
    "covidOccupiedMVBeds",
    # "hospitalCases",
    # "newAdmissions",
    # "new_confirmed",
    # "new_deceased",
    # "hospitalCases_rolling_7_mean",
    # "hospitalCases_rolling_7_std",
    # "newAdmissions_rolling_7_mean",
    # "newAdmissions_rolling_7_std",
    # "new_confirmed_rolling_7_mean",
    # "new_confirmed_rolling_7_std",
    # "new_deceased_rolling_7_mean",
    # "new_deceased_rolling_7_std",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
]

In [22]:
sample_df = sample_df[columns_to_select]
sample_df.head()

,covidOccupiedMVBeds,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,
2020-04-14,32.840992,467.0,485.0,485.0,475.0,469.0,445.0,360.0,4.0,14.0,1.0
2020-04-15,33.018215,467.0,467.0,485.0,475.0,469.0,450.0,360.0,4.0,15.0,2.0
2020-04-15,33.018215,472.0,467.0,467.0,485.0,475.0,450.0,406.0,4.0,15.0,2.0
2020-04-16,33.090706,472.0,472.0,467.0,485.0,475.0,465.0,406.0,4.0,16.0,3.0
2020-04-16,33.090706,474.0,472.0,472.0,467.0,485.0,465.0,419.0,4.0,16.0,3.0


In [23]:
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]

In [24]:
target = "covidOccupiedMVBeds"
pred_df = pd.concat([train[[target]], val[[target]]])

In [25]:
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

# Check a few batches from the training dataloader
train_loader = datamodule.train_dataloader()
for x, y in train_loader:
    print("Input batch shape:", x.shape)
    print("Output batch shape:", y.shape)
    break

Input batch shape: torch.Size([32, 7, 11])
Output batch shape: torch.Size([32, 1, 1])


In [26]:
# Setting up TensorBoard logger
# logger = TensorBoardLogger("notebook/multivariate/tb_logs", name="my_rnn_experiment")

# %tensorboard --logdir tb_logs


rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=11,  # 25 for multivariate time series
    hidden_size=32,  # hidden size of the RNN
    num_layers=5, # number of layers
    bidirectional=False, # bidirectional RNN
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
# model.float()

trainer = pl.Trainer(
    # logger=logger,
    min_epochs=5,
    max_epochs=100,
    accelerator = "gpu",
    devices = 1,
    callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
)
trainer.fit(model, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 9.9 K 
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
9.9 K     Trainable params
0         Non-trainable params
9.9 K     Total params
0.040     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
metric_record = []

In [ ]:
predictions = trainer.predict(model, datamodule.test_dataloader())
predictions = torch.cat(predictions).squeeze().detach().numpy()
# De-normalizing the predictions
predictions = predictions * datamodule.train.std + datamodule.train.mean

actuals = test["covidOccupiedMVBeds"].values

assert (
    actuals.shape == predictions.shape
), "Mismatch in shapes between actuals and predictions"

algorithm_name = rnn_config.rnn_type

metrics = {
    "Algorithm": algorithm_name,
    "MAE": mae(actuals, predictions),
    "MSE": mse(actuals, predictions),
    "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
    "Forecast Bias": forecast_bias(actuals, predictions),
}

value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
metrics = {
    key: format_.format(value)
    for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
}

pred_df_ = pd.DataFrame({f"Vanilla {algorithm_name}": predictions}, index=test.index)
pred_df = test.join(pred_df_)

metric_record.append(metrics)
print(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 7/7 [00:00<00:00, 306.43it/s]
{'Algorithm': 'RNN', 'MAE': '1.6175', 'MSE': '2.8424', 'MASE': '7.0797', 'Forecast Bias': '-1.62'}


In [ ]:
fig = plot_forecast(
    pred_df,
    forecast_columns=[f"Vanilla {algorithm_name}"],
    forecast_display_names=[f"Vanilla {algorithm_name}"],
)
title = f"Forecasting COVID-19 MVBeds with {algorithm_name}"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list(
            [
                # 3 days after training data starts from the first date in testing data
                dict(count=3, label="3D", step="day", stepmode="backward"),
                # 7 days after training data starts from the first date in testing data
                dict(count=7, label="1W", step="day", stepmode="backward"),
                # 14 days after training data starts from the first date in testing data
                dict(count=14, label="2W", step="day", stepmode="backward"),
                # 1 month after training data starts from the first date in testing data
                dict(count=1, label="1M", step="month", stepmode="backward"),
                # 3 months after training data starts from the first date in testing data
                dict(count=3, label="3M", step="month", stepmode="backward"),
                # all
                dict(step="all"),
            ]
        )
    ),
    type="date", # date range
    tickformat="%d-%m-%Y", # date format
    range=["2021-10-01", "2021-12-30"], # date range
)
# save_path = f"images/forecast_multivarate_{algorithm_name}.png"
# pio.write_image(fig, save_path)
fig.show()

In [ ]:
# plot the forecast of the training data and test data
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=train.index, y=train["covidOccupiedMVBeds"], mode="lines", name="Train Data"
    )
)
fig.add_trace(
    go.Scatter(
        x=val.index, y=val["covidOccupiedMVBeds"], mode="lines", name="Validation Data"
    )
)
fig.add_trace(
    go.Scatter(
        x=test.index, y=test["covidOccupiedMVBeds"], mode="lines", name="Test Data"
    )
)
# add a line to when the train ends and the val begins and also when the val ends and the test begins
fig.add_shape(
    dict(
        type="line",
        x0=train.index.max(),
        y0=0,
        x1=train.index.max(),
        y1=100,
        line=dict(color="black", width=2),
    )
)
fig.add_shape(
    dict(
        type="line",
        x0=val.index.max(),
        y0=0,
        x1=val.index.max(),
        y1=100,
        line=dict(color="black", width=2),
    )
)
fig.update_layout(
    title="Train, Validation and Test Data for COVID-19 Occupied MV Beds in Midlands Region",
    xaxis_title="Date",
    yaxis_title="COVID-19 Occupied MV Beds",
    template="plotly_white",
)

fig.show()


In [ ]:
fig = plot_forecast(
    pred_df,
    forecast_columns=[f"Vanilla {algorithm_name}"],
    forecast_display_names=[f"Vanilla {algorithm_name}"],
)
title = f"Forecasting COVID-19 MVBeds with {algorithm_name}"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    type="date", range=["2021-09-26", "2021-12-31"], dtick="M1", tickformat="%b %Y"
)
save_path = f"figures/forecast_multivarate_{algorithm_name}.pdf"
pio.write_image(fig, save_path)
fig.show()

In [32]:
# save model to reuse later, in the report folder 
model_path = Path("models")
model_path.mkdir(exist_ok=True)
model_file = model_path / f"{algorithm_name}_model.pt"
torch.save(model.state_dict(), model_file)

# save the metrics to a csv file
metric_df = pd.DataFrame(metric_record)
metric_file = model_path / f"{algorithm_name}_metrics.csv"

metric_df.to_csv(metric_file, index=False)

## Simulated annealing hyperparameter tunning

In [33]:
# Define the bounds for parameters
param_bounds = {
    "rnn_type": ["RNN", "GRU", "LSTM"],
    "hidden_size": (32, 128),  # Hidden size between 32 and 128
    "num_layers": (5, 30),
    "bidirectional": [True, False]
}

# Initial hyperparameters and temperature
initial_params = ["RNN", 64, 10, True]  # Updated for a realistic hidden size initialization
initial_temp = 10

In [34]:
# Define the objective function
def objective(params):
    rnn_type, hidden_size, num_layers, bidirectional = params
    rnn_config = SingleStepRNNConfig(
        rnn_type=rnn_type,
        input_size=11,
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional,
        learning_rate=1e-3
    )
    model = SingleStepRNNModel(rnn_config)
    model.float()

    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )
    trainer.fit(model, datamodule)
    
    shutil.rmtree("lightning_logs")

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean

    actuals = test["covidOccupiedMVBeds"].values

    assert (
        actuals.shape == predictions.shape
    ), "Mismatch in shapes between actuals and predictions"

    return np.mean(np.abs(actuals - predictions))  # Return the MAE


In [35]:
def neighbor(params):
    rnn_type, hidden_size, num_layers, bidirectional = params

    # Perturbations
    hidden_size = np.random.randint(*param_bounds["hidden_size"])
    num_layers = np.random.randint(*param_bounds["num_layers"])
    # learning_rate = np.random.uniform(*param_bounds["learning_rate"])
    rnn_type = np.random.choice(param_bounds["rnn_type"])
    bidirectional = bool(
        np.random.choice(param_bounds["bidirectional"])
    )  # Convert to native boolean
    
    # ensure that it returns the best parameters
    return [rnn_type, hidden_size, num_layers, bidirectional]

# Simulated Annealing Algorithm, measuring cost and temperature
def simulated_annealing(objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate=0.20, verbose=True):
    current_params = initial_params
    current_cost = objective(current_params)
    best_params = current_params
    best_cost = current_cost
    temp = initial_temp
    cost_history = []

    for i in range(n_iter):
        candidate_params = neighbor(current_params)
        candidate_cost = objective(candidate_params)

        # Calculate the probability of accepting the new solution
        acceptance_probability = np.exp(-abs(candidate_cost - current_cost) / temp)

        # Decision to accept the new candidate
        if candidate_cost < current_cost or np.random.uniform() < acceptance_probability:
            current_params = candidate_params
            current_cost = candidate_cost

            # Update the best found solution
            if current_cost < best_cost:
                best_params = current_params
                best_cost = current_cost

        # Cooling down the temperature
        temp *= cooling_rate
        cost_history.append(best_cost)

        # Output current iteration details
        if verbose:
            print(f"Iteration: {i+1}, Best Cost: {best_cost:.4f}, Current Cost: {current_cost:.4f}, Temperature: {temp:.4f}")

        # Break early if the minimum cost has been constant for 5 iterations
        if i >= 5 and all(x == best_cost for x in cost_history[-5:]):
            print("Early stopping as there is no improvement in the last 5 iterations.")
            break

    return best_cost, best_params, cost_history

In [36]:
# Run Simulated Annealing for 100 iterations
initial_params = ["RNN", 64, 10, True]  # Initial parameters
initial_temp = 10
n_iter = 100
cooling_rate = 0.95  # More gradual cooling

best_cost, best_params, cost_history = simulated_annealing(
    objective, initial_params, initial_temp, neighbor, n_iter, cooling_rate
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 233 K 
1 | fc   | Linear  | 129   
2 | loss | MSELoss | 0     
---------------------------------
233 K     Trainable params
0         Non-trainable params
233 K     Total params
0.934     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Iteration: 8, Best Cost: 0.9188, Current Cost: 3.2343, Temperature: 6.6342
Early stopping as there is no improvement in the last 5 iterations.


In [37]:
# Print the best parameters and best cost
print(f"Best Parameters: {best_params}, Best Cost: {best_cost}")

Best Parameters: ['GRU', 84, 6, False], Best Cost: 0.9187866660539992


In [38]:
# plot the MAEs gotten from the SA
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, len(cost_history) + 1), y=cost_history, mode="lines"))
fig.update_layout(
    title="Simulated Annealing Optimization for Hyperparameter Tuning",
    xaxis_title="Iteration",
    yaxis_title="Best Cost",
    template="plotly_white",
)
fig.show()

In [39]:
# prediction using the best parameters
rnn_config = SingleStepRNNConfig(
    rnn_type=best_params[0],
    input_size=11,
    hidden_size=best_params[1],
    num_layers=best_params[2],
    bidirectional=best_params[3],
    learning_rate=1e-3,
)

model = SingleStepRNNModel(rnn_config)
# model.float()

# logger = TensorBoardLogger("notebook/multivariate/tb_logs", name="my_rnn_experiment")
trainer = pl.Trainer(
    # logger=logger,
    min_epochs=5,
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
)

trainer.fit(model, datamodule)

predictions = trainer.predict(model, datamodule.test_dataloader())
predictions = torch.cat(predictions).squeeze().detach().numpy()
predictions = predictions * datamodule.train.std + datamodule.train.mean

actuals = test["covidOccupiedMVBeds"].values

assert (
    actuals.shape == predictions.shape
), "Mismatch in shapes between actuals and predictions"

algorithm_name = rnn_config.rnn_type

metrics = {
    "Algorithm": f"{algorithm_name} (SA)",
    "MAE": mae(actuals, predictions),
    "MSE": mse(actuals, predictions),
    "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
    "Forecast Bias": forecast_bias(actuals, predictions),
}

value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]

metrics = {
    key: format_.format(value)
    for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
}

pred_df_ = pd.DataFrame({f"Optimized {algorithm_name} (SA)": predictions}, index=test.index)

pred_df = test.join(pred_df_)

metric_record.append(metrics)

print(metrics)

fig = plot_forecast(
    pred_df,
    forecast_columns=[f"Optimized {algorithm_name} (SA)"],
    forecast_display_names=[f"Optimized {algorithm_name} (SA)"],
)

title = f"Forecasting COVID-19 MVBeds with {algorithm_name} (SA)"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list(
            [
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        )
    ),
)

# save_path = f"images/forecast_multivarate_{algorithm_name}.png"
# pio.write_image(fig, save_path)
fig.show()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | GRU     | 238 K 
1 | fc   | Linear  | 85    
2 | loss | MSELoss | 0     
---------------------------------
238 K     Trainable params
0         Non-trainable params
238 K     Total params
0.955     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

{'Algorithm': 'GRU (SA)', 'MAE': '1.1496', 'MSE': '1.7012', 'MASE': '5.0318', 'Forecast Bias': '-1.14'}


In [40]:
# shutil.rmtree("notebook/multivariate/tb_logs")

In [41]:
# Plotting the forecast
fig = plot_forecast(
    # simulated annealing model
    pred_df,
    forecast_columns=[f"Optimized {algorithm_name} (SA)"],
    forecast_display_names=[f"Optimized {algorithm_name} (SA)"],
    # forecast_columns=[f"{algorithm_name} (SA)"],
    # forecast_display_names=[algorithm_name],
)

title = f"Forecasting multivarate COVID-19 MVBeds with {algorithm_name} (SA)"
fig = format_plot(fig, title=title)

fig.update_xaxes(
    type="date", range=["2021-09-26", "2021-12-31"], dtick="M1", tickformat="%b %Y"
)

save_path = f"images/forecast_multivarate_{algorithm_name}_sa.png"
# pio.write_image(fig, save_path)
fig.show()

In [42]:
# save model to reuse later, in the report folder
model_file = model_path / f"{algorithm_name}_sa_model.pt"
torch.save(model.state_dict(), model_file)

# save the metrics to a csv file
metric_file = model_path / f"{algorithm_name}_sa_metrics.csv"
metric_df.to_csv(metric_file, index=False)

In [43]:
HORIZON = 1
WINDOW = 7

In [51]:
encoder_config = RNNConfig(
    input_size=11,  # Replace with actual number of input features
    hidden_size=32,  # Example size
    num_layers=5,
    bidirectional=True
)

decoder_config = RNNConfig(
    input_size=11,  # Should align with the encoder's output dimension
    hidden_size=32,  # Example size
    num_layers=5,
    bidirectional=True
)

rnn2fc_config = Seq2SeqConfig(
    encoder_type="GRU",
    decoder_type="FC",
    encoder_params=encoder_config,
    decoder_params={"window_size": WINDOW, "horizon": HORIZON},
    decoder_use_all_hidden=True,
    learning_rate=1e-3,
)

model = Seq2SeqModel(rnn2fc_config)

trainer = pl.Trainer(
    # logger=logger,
    min_epochs=5,
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
)

trainer.fit(model, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | encoder | GRU     | 83.9 K
1 | decoder | Linear  | 449   
2 | loss    | MSELoss | 0     
------------------------------------
84.4 K    Trainable params
0         Non-trainable params
84.4 K    Total params
0.337     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [52]:
tag = f"{rnn2fc_config.encoder_type}_{rnn2fc_config.decoder_type}_{'all_hidden' if rnn2fc_config.decoder_use_all_hidden else 'last_hidden'}"

predictions = trainer.predict(model, datamodule.test_dataloader())
predictions = torch.cat(predictions).squeeze().detach().numpy()

predictions = predictions * datamodule.train.std + datamodule.train.mean

actuals = test["covidOccupiedMVBeds"].values

assert (
    actuals.shape == predictions.shape
), "Mismatch in shapes between actuals and predictions"

algorithm_name = rnn2fc_config.encoder_type

metrics = {
    "Algorithm": f"Seq2Seq {algorithm_name}",
    "MAE": mae(actuals, predictions),
    "MSE": mse(actuals, predictions),
    "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
    "Forecast Bias": forecast_bias(actuals, predictions),
}

value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]

metrics = {
    key: format_.format(value)
    for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
}

pred_df_ = pd.DataFrame({f"Seq2Seq {algorithm_name}": predictions}, index=test.index)

pred_df = test.join(pred_df_)

metric_record.append(metrics)

print(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

{'Algorithm': 'Seq2Seq GRU', 'MAE': '1.6156', 'MSE': '2.8694', 'MASE': '7.0713', 'Forecast Bias': '-1.62'}


In [53]:
fig = plot_forecast(
    pred_df,
    forecast_columns=[f"Seq2Seq {algorithm_name}"],
    forecast_display_names=[f"Seq2Seq {algorithm_name}"],
)

title = f"Forecasting COVID-19 MVBeds with Seq2Seq {algorithm_name}"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list(
            [
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        )
    ),
)

# save_path = f"images/forecast_multivarate_{algorithm_name}.png"
# pio.write_image(fig, save_path)
fig.show()

In [54]:
metric_df = pd.DataFrame(metric_record)
metric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Algorithm      5 non-null      object
 1   MAE            5 non-null      object
 2   MSE            5 non-null      object
 3   MASE           5 non-null      object
 4   Forecast Bias  5 non-null      object
dtypes: object(5)
memory usage: 328.0+ bytes


In [55]:
metric_df[["MAE", "MSE", "MASE", "Forecast Bias"]] = metric_df[
    ["MAE", "MSE", "MASE", "Forecast Bias"]
].astype("float32")

In [56]:
formatted = metric_df.style.format(
    {
        "MAE": "{:.4f}",
        "MSE": "{:.4f}",
        "MASE": "{:.4f}",
        "Forecast Bias": "{:.2f}%",
        "Time Elapsed": "{:.6f}",
    }
)
formatted = formatted.highlight_min(
    color="lightgreen", subset=["MAE", "MSE", "MASE"]
).apply(
    highlight_abs_min,
    props="color:black;background-color:lightgreen",
    axis=0,
    subset=["Forecast Bias"],
)
formatted

,Algorithm,MAE,MSE,MASE,Forecast Bias
0,RNN,1.6175,2.8424,7.0797,-1.62%
1,GRU (SA),1.1496,1.7012,5.0318,-1.14%
2,LSTM,2.1975,5.1047,9.6186,-2.20%
3,Seq2Seq LSTM,2.1975,5.1047,9.6186,-2.20%
4,Seq2Seq GRU,1.6156,2.8694,7.0713,-1.62%
